### nudging tendency of moisture, physics and dynamics components

Compute and plot physics and dynamics contribution to < dQ2 >, averaged over the last 36 days of the nudged training run

Adapted from https://github.com/VulcanClimateModeling/explore/blob/master/spencerc/2021-02-15-n2f-vs-hybrid-dQ2/2021-04-06-dQ1-dQ2-comparison.ipynb

<!-- Depends on this branch in fv3net: https://github.com/VulcanClimateModeling/fv3net/tree/update-hybrid, only for the update to the `open_fine_resolution_nudging_hybrid` function.  Averaging period: 20160803.014500 to 20160909.224500 -->

In [ ]:
import os
import intake
import cartopy.crs as ccrs
import cftime
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update(**{
    'mathtext.fontset': 'stix'
})
import numpy as np
from datetime import timedelta
import xarray as xr
import fv3viz
from vcm.fv3 import metadata
from vcm.catalog import catalog as CATALOG
from loaders.mappers._fine_resolution_budget import FineResolutionSources
from loaders.mappers import XarrayMapper
from dask.diagnostics import ProgressBar

In [ ]:
GRID = metadata.standardize_fv3_diagnostics(CATALOG["grid/c48"].to_dask())
N2F_URL = "gs://vcm-ml-experiments/2021-04-13-n2f-c3072/3-hrly-ave-rad-precip-setting-30-min-rad-timestep-shifted-start-tke-edmf"
FINE_RES_URL = 'gs://vcm-ml-experiments/default/2021-04-27/2020-05-27-40-day-X-SHiELD-simulation/fine-res-budget.zarr'
VERIF_ENTRY = "40day_c48_atmos_8xdaily_may2020"
SECONDS_PER_DAY=86400
OUTDIR = 'figures'

In [ ]:
# load nudge-to-fine run dataset
nudging_tendencies = metadata.standardize_fv3_diagnostics(
    intake.open_zarr(os.path.join(N2F_URL, 'nudging_tendencies.zarr'), consolidated=True).to_dask()
)
physics_tendencies = metadata.standardize_fv3_diagnostics(
    intake.open_zarr(os.path.join(N2F_URL, 'physics_tendencies.zarr'), consolidated=True).to_dask()
)
dycore = metadata.standardize_fv3_diagnostics(
    intake.open_zarr(os.path.join(N2F_URL, 'atmos_dt_atmos.zarr'), consolidated=True).to_dask()
)
state = metadata.standardize_fv3_diagnostics(
        intake.open_zarr(os.path.join(N2F_URL, 'state_after_timestep.zarr'), consolidated=True).to_dask(),
)
nudged_run = xr.merge([nudging_tendencies, physics_tendencies, dycore, state], join='inner')

In [ ]:
# subset time to exclude first four spinup days of the simulation
nudged_run = nudged_run.sel(time=nudged_run.time.loc[nudged_run.time > cftime.DatetimeJulian(2016, 8, 5, 0, 0, 0, 0)])

In [ ]:
# downsample 15-minute data to 3-hourly nudging tendency output using nearest-neighbor approach
# could time-average but that is ~12x slower with probably little accuracy change
def time_downsample(ds, freq='3H'):
    ds = ds.resample({'time': freq}, base=1, loffset=timedelta(minutes=90)).nearest()
    return ds

In [ ]:
# load fine-res apparent sources using mapper approach and fine-res zarr
fine_res_ds = time_downsample(intake.open_zarr(FINE_RES_URL).to_dask())
fine_res_ds = fine_res_ds.sel(time=fine_res_ds.time.loc[fine_res_ds.time > (nudged_run.time[0] - timedelta(minutes=90))])
fine_res_mapper = XarrayMapper(fine_res_ds)
RENAME = {
    'pfull': 'z',
    'dQ1': 'Q1',
    'dQ2': 'Q2',
    'omega': 'omega_fine_rs'
}
DIM_ORDER = ("tile", "z", "grid_yt", "grid_xt")
fine_res_sources_mapper = FineResolutionSources(fine_res_mapper, rename_vars=RENAME, dim_order=DIM_ORDER)
Q2 = xr.concat(
    [fine_res_sources_mapper[key]['Q2'] for key in fine_res_sources_mapper.keys()],
    dim='time'
).assign_coords({'time': nudged_run.time}).rename({'grid_yt': 'y', 'grid_xt': 'x'})

In [ ]:
# compute components of moisture nudging tendency
dQ2p = (Q2 - nudged_run.tendency_of_specific_humidity_due_to_fv3_physics).assign_attrs({'long_name': 'specific humidity nudging due to physics differences', 'units': 'kg/kg/s'})
dQ2d = (nudged_run.specific_humidity_tendency_due_to_nudging - dQ2p).assign_attrs({'long_name': 'specific humidity nudging due to dynamics differences', 'units': 'kg/kg/s'})
tendencies = {
    'dQ2': nudged_run.specific_humidity_tendency_due_to_nudging,
    'dQ2p': dQ2p,
    'dQ2d': dQ2d
}

In [ ]:
def vertical_integral_of_moisture_tendency(da, delp):
    return (SECONDS_PER_DAY*((da * delp).sum("z") / 9.81)).assign_attrs(
        {
            'long_name': 'column integral of ' + da.attrs.get('long_name', ''),
            'units': 'mm/day'
        }
    )

In [ ]:
# column-integrate and time-average
column_integrated_ds = {}
for var in tendencies:
    column_integrated_ds[f"column_integrated_{var}"] = vertical_integral_of_moisture_tendency(
        tendencies[var], nudged_run['pressure_thickness_of_atmospheric_layer']
    )
    
with ProgressBar():
    column_integrated_ds = xr.Dataset(column_integrated_ds).mean(dim='time').load()

In [ ]:
# get vertical wind difference between coarse and fine
vertical_wind_nudged_run = nudged_run.w500
vertical_wind_fine_res = metadata.standardize_fv3_diagnostics(CATALOG[VERIF_ENTRY].to_dask()).w500.interp({'time': nudged_run.time})
vertical_wind_difference = (100*(vertical_wind_fine_res - vertical_wind_nudged_run)).assign_attrs({'long_name': 'vertical wind difference, fine - coarse', 'units': 'cm/s'}).rename('vertical_wind_difference')

with ProgressBar():
    vertical_wind_difference = vertical_wind_difference.mean("time", keep_attrs=True).compute()

In [ ]:
MAPPABLE_VAR_KWARGS = {
    "coord_x_center": "x",
    "coord_y_center": "y",
    "coord_x_outer": "x_interface",
    "coord_y_outer": "y_interface",
    "coord_vars": {
        "lonb": ["y_interface", "x_interface", "tile"],
        "latb": ["y_interface", "x_interface", "tile"],
        "lon": ["y", "x", "tile"],
        "lat": ["y", "x", "tile"],
    },
}

In [ ]:
fig, axes = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.Robinson()))
axes = axes.flatten()
ax0 = axes[0]
dQ2 = fv3viz.mappable_var(xr.merge([column_integrated_ds.column_integrated_dQ2, GRID]), 'column_integrated_dQ2', **MAPPABLE_VAR_KWARGS)
h0 = fv3viz.plot_cube_axes(
    dQ2.column_integrated_dQ2.values,
    dQ2.lat.values,
    dQ2.lon.values,
    dQ2.latb.values,
    dQ2.lonb.values,
    "pcolormesh",
    vmin=-4,
    vmax=4,
    ax=ax0,
    cmap='RdBu_r'
)
ax0.set_title(
    r"a) $\langle \Delta Q_q \rangle $" + f", mean: {column_integrated_ds.column_integrated_dQ2.weighted(GRID['area']).mean(['x', 'y', 'tile']).compute().item():1.2f}"
)
ax0.coastlines()
plt.colorbar(h0, ax=ax0, label="mm/day")
ax1 = axes[1]
dQ2d = fv3viz.mappable_var(xr.merge([column_integrated_ds.column_integrated_dQ2d, GRID]), 'column_integrated_dQ2d', **MAPPABLE_VAR_KWARGS)
h1 = fv3viz.plot_cube_axes(
    dQ2d.column_integrated_dQ2d.values,
    dQ2d.lat.values,
    dQ2d.lon.values,
    dQ2d.latb.values,
    dQ2d.lonb.values,
    "pcolormesh",
    vmin=-4,
    vmax=4,
    ax=ax1,
    cmap='RdBu_r'
)
ax1.set_title(
    r"b) $\langle \Delta Q_q^d \rangle$" + f", mean: {column_integrated_ds.column_integrated_dQ2d.weighted(GRID['area']).mean(['x', 'y', 'tile']).compute().item():1.2f}"
)
ax1.coastlines()
plt.colorbar(h1, ax=ax1, label="mm/day")
ax2 = axes[2]
dQ2p = fv3viz.mappable_var(xr.merge([column_integrated_ds.column_integrated_dQ2p, GRID]), 'column_integrated_dQ2p', **MAPPABLE_VAR_KWARGS)
h2 = fv3viz.plot_cube_axes(
    dQ2p.column_integrated_dQ2p.values,
    dQ2p.lat.values,
    dQ2p.lon.values,
    dQ2p.latb.values,
    dQ2p.lonb.values,
    "pcolormesh",
    vmin=-4,
    vmax=4,
    ax=ax2,
    cmap='RdBu_r'
)
ax2.set_title(
    r"c) $\langle \Delta Q_q^p \rangle$" + f", mean: {column_integrated_ds.column_integrated_dQ2p.weighted(GRID['area']).mean(['x', 'y', 'tile']).compute().item():1.2f}"
)
ax2.coastlines()
plt.colorbar(h2, ax=ax2, label="mm/day")
ax3 = axes[3]
dW = fv3viz.mappable_var(xr.merge([vertical_wind_difference, GRID]), 'vertical_wind_difference', **MAPPABLE_VAR_KWARGS)
h3 = fv3viz.plot_cube_axes(
    dW.vertical_wind_difference.values,
    dW.lat.values,
    dW.lon.values,
    dW.latb.values,
    dW.lonb.values,
    "pcolormesh",
    vmin=-1,
    vmax=1,
    ax=ax3,
    cmap='RdBu_r'
)
ax3.set_title(
    f"d) w500 difference, mean: {dW.vertical_wind_difference.weighted(GRID['area']).mean(['x', 'y', 'tile']).compute().item():1.2f}"
)
ax3.coastlines()
plt.colorbar(h3, ax=ax3, label="cm/s")
fig.set_size_inches([10, 6])
fig.set_dpi(150)
fig.tight_layout()
fig.savefig(f"{OUTDIR}/column_integrated_moisture_tendency_components.png")